In [1]:
from sklearn.cluster import HDBSCAN
from alpha_shapes import Alpha_Shaper
from polylabel import polylabel
import json

ALPHA = 0.5

In [2]:
with open('public/data/Official_Neighborhoods.geojson') as file:
  neighborhoods = json.load(file)
features = neighborhoods['features']
for feature in features:
  if feature['geometry']['type'] == 'MultiPolygon':
    center_points = []
    for subfeature in feature['geometry']['coordinates']:
      polygon = subfeature[0]
      shaper = Alpha_Shaper(polygon)
      alpha_shape = shaper.get_shape(ALPHA)
      alpha_shape_coords = [list(coord) for coord in alpha_shape.exterior.coords]
      center_point = polylabel([alpha_shape_coords])
      center_points.append([float(center_point[0]), float(center_point[1])])
    feature['geometry'] = {
      "type": "MultiPoint",
      "coordinates": center_points
    }
      
  elif feature['geometry']['type'] == 'Polygon':
    polygon = feature['geometry']['coordinates'][0]
    shaper = Alpha_Shaper(polygon)
    alpha_shape = shaper.get_shape(ALPHA)
    alpha_shape_coords = [list(coord) for coord in alpha_shape.exterior.coords]
    center_point = polylabel([alpha_shape_coords])
    feature['geometry'] = {
      "type": "Point",
      "coordinates": [float(center_point[0]), float(center_point[1])]
    }
  else:
    print(feature['geometry']['type'])
    print(feature['properties']['NAME'])

with open('public/data/Official_Neighborhoods_Centers.geojson', 'w') as file:
  json.dump(neighborhoods, file)